# HOP-4: same day patient count <= same day transaction count

Description: check if all the rows having same day patient count are less and equal to same day transaction count.

Starting Author: Amy Jin (amy@careset.com)

Date: July 20th, 2018

https://docs.google.com/spreadsheets/d/1sltXNV5CDfkClTs23rCgOSImqeZDIjJJoTZQwtDvZ18/edit#gid=0

## Connection to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [4]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                       database name in server
# 2）table_name:                    table name
# 3) same_day_patient_count:        same day patient count column 
# 4) same_day_transaction_count:    same day transaction count column
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, the test will print out all the same_day_patient_count and 
# same_day_transaction_count that fail the test.


def hop_4(db_name, table_name, same_day_patient_count, same_day_transaction_count):

    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()

        # Use `DISTINCT` to reduce the calculation load
        query = ('''
            SELECT *
            FROM {db}.{t1}
            WHERE {col1} > {col2}
            LIMIT 1;
        '''.format(db = db_name, t1 = table_name, col1 = same_day_patient_count, col2 = same_day_transaction_count))

        cur.execute(query)
        rows = list(sum(cur.fetchall(), ()))

        if not len(rows):
            print ("Count-7 test result for {}.{} : PASS".format(db_name, table_name))
        else:                   
            print ("Count-7 test result for {}.{} : FAIL".format(db_name, table_name) + '\n')
            print ("The following {}, {} fail the test:".format(same_day_patient_count, same_day_transaction_count) + '\n')

            # MySQL query to get the failed rows
            query = ('''
                SELECT {col1}, {col2}
                FROM {db}.{t1}
                WHERE {col1} > {col2};
            '''.format(db = db_name, t1 = table_name, col1 = same_day_patient_count, col2 = same_day_transaction_count))

            cur.execute(query)

            for row in cur.fetchall():
                for i in range(0,len(row)):
                    print (str(row[i]), end=", ")
                print ('\n')

        cur.close()
        connection.close()

## Test Example

In [5]:
hop_4('_amy', 'test_hop_directed_good', 'same_day_patient_count', 'same_day_transaction_count')

Count-7 test result for _amy.test_hop_directed_good : PASS


In [6]:
hop_4('_amy', 'test_hop_directed_bad', 'same_day_patient_count', 'same_day_transaction_count')

Count-7 test result for _amy.test_hop_directed_bad : FAIL

The following same_day_patient_count, same_day_transaction_count fail the test:

500, 218, 

500, 32, 

500, 46, 

500, 115, 

500, 21, 

500, 12, 

500, 13, 

500, 180, 

500, 12, 

500, 21, 

500, 39, 

500, 46, 

500, 0, 

500, 0, 

500, 0, 

500, 0, 

500, 0, 

500, 0, 

500, 0, 

500, 0, 

500, 0, 

500, 0, 

500, 11, 

500, 0, 

500, 0, 

500, 0, 

500, 0, 

500, 0, 

500, 61, 

